In [45]:
!pip install praw

In [46]:
import pandas as pd
import praw
import nltk
import matplotlib.pyplot as plt

In [47]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')

def analyze_sentiment(comment_text):
    sid = SentimentIntensityAnalyzer()
    sentiment_score = sid.polarity_scores(comment_text)['compound']
    scaled_sentiment = sentiment_score
    return scaled_sentiment

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [48]:
def collect_and_analyze_data(reddit, post_url):
    submission = reddit.submission(url=post_url)

    post_data = {
        "Título": submission.title,
        "Texto do Post": submission.selftext,
        "Sentimento": 1 if analyze_sentiment(submission.selftext) > 0 else -1,
        "ID do Comentário": submission.id,
        "Link do Post": post_url,
        "Comentários": []
    }

    submission.comments.replace_more(limit=1)
    comments = submission.comments.list()

    comment_id_to_data = {}

    for comment in comments:
        comment_id = comment.id
        comment_data = {
            "ID do Comentário": comment_id,
            "Usuário": comment.author.name if comment.author else "Autor Desconhecido",
            "Texto": comment.body,
            "Sentimento": analyze_sentiment(comment.body),
            "Responde_a": submission.id,
            "Interação": submission.id,
            "Replies": []
        }

        parent_id = comment.parent_id.split('_')[-1]
        if parent_id in comment_id_to_data:
            comment_data["Responde_a"] = comment_id_to_data[parent_id]['ID do Comentário']
            comment_data["Interação"] = comment_id_to_data[parent_id]['Usuário']
            comment_id_to_data[parent_id]["Replies"].append(comment_data)
        else:
            post_data["Comentários"].append(comment_data)

        comment_id_to_data[comment_id] = comment_data

    return post_data

CLIENT_ID = 'mG-tLZ2hh_ij1TmEzLMKxA'
SECRET_KEY = 'i7lEZcbQWDyDscuvxCOkVb0rPpWf-g'
user_agent = 'MyAPI/0.0.1'
username = 'bryan_limaC'
password = 'gerson159951'

reddit = praw.Reddit(
    client_id=CLIENT_ID,
    client_secret=SECRET_KEY,
    user_agent=user_agent,
    username=username,
    password=password
)

post_url = 'https://www.reddit.com/r/PoliticalDiscussion/comments/18650ll/what_options_can_be_used_to_make_parties_more/'
dados = collect_and_analyze_data(reddit, post_url)
Conjunto_Dados_Usuario = pd.DataFrame([dados])

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



In [49]:
import pandas as pd


def coletar_respostas(comment, conjuntos_comentarios):

    id = comment['ID do Comentário']
    usuario = comment['Usuário']
    texto = comment['Texto']
    responde_a = comment['Responde_a']
    sentimento = comment['Sentimento']
    inter = comment['Interação']
    replies = comment['Replies']

    if usuario != 'Autor Desconhecido':
      if replies:
          for reply in replies:
              coletar_respostas(reply, conjuntos_comentarios)
      nova_linha = {'Usuário': usuario, 'C1': id, 'C2': responde_a, 'W': sentimento, 'Interação': inter}
      conjuntos_comentarios.append(nova_linha)


usuarios_comentarios = pd.DataFrame(columns=['Usuário', 'C1', 'C2', 'W', 'Interação', 'Num_resposta'])
conjuntos_comentarios = []

for index, row in Conjunto_Dados_Usuario.iterrows():
    comentario = row['Comentários']
    for comment in comentario:
        coletar_respostas(comment, conjuntos_comentarios)
usuarios_comentarios = usuarios_comentarios.append(conjuntos_comentarios, ignore_index=True)


<ipython-input-49-c872b84c911b>:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  usuarios_comentarios = usuarios_comentarios.append(conjuntos_comentarios, ignore_index=True)


In [50]:
usuarios_comentarios['Num_resposta'] = 0
user_num_resposta = 0
for index, row in usuarios_comentarios.iterrows():
  user_num_resposta = usuarios_comentarios.at[index, 'Num_resposta']
  for index2, row2 in usuarios_comentarios.iterrows():
    if index2 > index:
      if row['Interação'] == row2['Usuário']:
        usuarios_comentarios.at[index2, 'Num_resposta'] = usuarios_comentarios.at[index2, 'Num_resposta'] + user_num_resposta + 1
        break

In [51]:
usuarios_comentarios['Sc1'] = ''
usuarios_comentarios['Sc2'] = ''

for index, row in usuarios_comentarios.iterrows():
    if row['C2'] == Conjunto_Dados_Usuario['ID do Comentário'][0]:
        usuarios_comentarios.at[index, 'Sc2'] = 1

In [52]:
tem_valor_vazio = True

while tem_valor_vazio:
    tem_valor_vazio = False

    for index, row in usuarios_comentarios.iterrows():
        if (row['Sc1'] == ''):
          tem_valor_vazio = True
          if(row['Sc2']):
            if (row['Sc2'] == 1 and row['W'] > 0) or (row['Sc2'] == -1 and row['W'] <= 0):
                usuarios_comentarios.at[index, 'Sc1'] = 1
                sc1_atual = 1
            else:
                usuarios_comentarios.at[index, 'Sc1'] = -1
                sc1_atual = -1

            for index2, row2 in usuarios_comentarios.iterrows():
                if row2['C2'] == row['C1']:
                    usuarios_comentarios.at[index2, 'Sc2'] = sc1_atual


In [53]:
S_ci = usuarios_comentarios.groupby("Usuário")["Sc1"].mean().reset_index()
engajamento = usuarios_comentarios.groupby("Usuário")["Num_resposta"].mean().reset_index()

In [54]:
usuarios_comentarios_positivos = usuarios_comentarios[usuarios_comentarios['W'] > 0]
usuarios_comentarios_negativo = usuarios_comentarios[usuarios_comentarios['W'] <= 0]

usuarios_comentarios_total = usuarios_comentarios.groupby(['Usuário','Interação']).mean()


grupo_positivo = usuarios_comentarios_positivos.groupby(['Usuário','Interação']).size().reset_index(name='quantidade_intercao_positiva')
grupo_negativo = usuarios_comentarios_negativo.groupby(['Usuário','Interação']).size().reset_index(name='quantidade_intercao_negativa')

grupo2 = usuarios_comentarios.groupby(["Usuário", "Interação"]).size().reset_index(name='quantidade_total_intercao')

grupo3 = pd.concat([grupo_positivo, grupo_negativo]).merge(grupo2,  left_on=['Usuário','Interação'], right_on=['Usuário','Interação'], how='left')


grupo3['p'] = grupo3['quantidade_intercao_positiva']/grupo3['quantidade_total_intercao']

grupo3 = grupo3.merge(usuarios_comentarios_total, left_on=['Usuário','Interação'], right_on=['Usuário','Interação'], how='left')
grupo3 = grupo3.fillna(0)


<ipython-input-54-49446089d04a>:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  usuarios_comentarios_total = usuarios_comentarios.groupby(['Usuário','Interação']).mean()


In [55]:
grupo3 = grupo3.drop(grupo3.loc[(grupo3['quantidade_intercao_negativa'] >= 1) & (grupo3['quantidade_intercao_negativa'] < grupo3['quantidade_total_intercao'])].index)

In [56]:
G = pd.DataFrame(columns=["Usuário", "Interação", "Sc(i)", "p", "w", "Engajamento"])
G['p'] = 0
G['w'] = 0

G["Usuário"] = grupo3["Usuário"]
G["Interação"] = grupo3["Interação"]

usuario_engajamento_mapping = engajamento.set_index('Usuário')['Num_resposta'].to_dict()
G["Engajamento"] = G['Usuário'].map(usuario_engajamento_mapping)
usuario_sc1_mapping = S_ci.set_index('Usuário')['Sc1'].to_dict()
G['Sc(i)'] = G['Usuário'].map(usuario_sc1_mapping)
G['p'] = grupo3['p']
G['w'] = grupo3['W']


In [57]:
G['Engajamento'] = G['Engajamento']/(G['Engajamento'].sum())


In [58]:
GrafoBipartido = pd.DataFrame(columns=["Usuário", "Karma"])
GrafoBipartido = G.filter(items=['Usuário']).drop_duplicates()

In [59]:
for index, row in GrafoBipartido.iterrows():
  user = row['Usuário']
  karma = reddit.redditor(user).comment_karma
  GrafoBipartido.at[index, 'Karma'] = karma

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/l

In [60]:
G['Influencia'] = 0
G['Influencia'] = G['Usuário'].map(G.groupby(['Usuário']).size())


In [61]:
G['Influencia'] = G['Influencia']/(G['Influencia'].sum())

In [62]:
karma_total = GrafoBipartido['Karma']/(GrafoBipartido['Karma'].sum())

In [63]:
GrafoBipartido['Karma'] = karma_total

In [64]:
novo_usuario = Conjunto_Dados_Usuario['ID do Comentário'][0]
novo_karma = 0

nova_linha = {'Usuário': novo_usuario, 'Karma': novo_karma}
GrafoBipartido = GrafoBipartido.append(nova_linha, ignore_index=True)



<ipython-input-64-5020e6b4d735>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  GrafoBipartido = GrafoBipartido.append(nova_linha, ignore_index=True)


In [65]:

import numpy as np

matriz_adjacencias = np.zeros((len(GrafoBipartido), len(GrafoBipartido)), dtype=int)

tabela_relacoes = []

for index, row in G.iterrows():
    tabela_relacoes.append((row['Usuário'], row['Interação']))


usuario_para_indice = {usuario: indice for indice, usuario in enumerate(GrafoBipartido['Usuário'])}

for origem, destino in tabela_relacoes:
    indice_origem = usuario_para_indice[origem]
    indice_destino = usuario_para_indice[destino]
    matriz_adjacencias[indice_origem][indice_destino] = 1
    matriz_adjacencias[indice_destino][indice_origem] = 1

data_df = pd.DataFrame(matriz_adjacencias)


In [66]:
import numpy as np

def brandes_betweenness_centrality(adjacency_matrix):
    num_nodes = len(adjacency_matrix)
    nodes = list(range(num_nodes))

    betweenness = {node: 0.0 for node in nodes}

    for source in nodes:
        stack = []
        predecessors = {node: [] for node in nodes}
        num_shortest_paths = {node: 0 for node in nodes}
        distance = {node: -1 for node in nodes}
        distance[source] = 0
        num_shortest_paths[source] = 1
        queue = [source]

        while queue:
            current_node = queue.pop(0)
            stack.append(current_node)

            for neighbor in nodes:
                if adjacency_matrix[current_node][neighbor] == 1:
                    if distance[neighbor] < 0:
                        queue.append(neighbor)
                        distance[neighbor] = distance[current_node] + 1

                    if distance[neighbor] == distance[current_node] + 1:
                        num_shortest_paths[neighbor] += num_shortest_paths[current_node]
                        predecessors[neighbor].append(current_node)

        dependency = {node: 0.0 for node in nodes}
        while stack:
            current_node = stack.pop()
            for predecessor in predecessors[current_node]:
                dependency[predecessor] += (num_shortest_paths[predecessor] / num_shortest_paths[current_node]) * (1 + dependency[current_node])
            if current_node != source:
                betweenness[current_node] += dependency[current_node]

    max_betweenness = max(betweenness.values())
    min_betweenness = min(betweenness.values())

    normalized_betweenness = {node: (value - min_betweenness) / (max_betweenness - min_betweenness) for node, value in betweenness.items()}

    return normalized_betweenness



betweenness_centrality = brandes_betweenness_centrality(matriz_adjacencias)
centrality_df = pd.DataFrame(list(betweenness_centrality.items()), columns=['Node', 'Betweenness Centrality'])



In [67]:
GrafoWhithCentrality = pd.merge(GrafoBipartido, centrality_df, left_index=True, right_index=True)
GrafoWhithCentrality.merge(G, left_on='Usuário', right_on='Usuário', how='left')

,Usuário,Karma,Node,Betweenness Centrality,Interação,Sc(i),p,w,Engajamento,Influencia
0,2manyfelines,0.021207,0,0.000000,KevinCarbonara,1.000000,1.0,0.58590,0.000000,0.002347
1,AutoModerator,0.000235,1,0.000000,18650ll,1.000000,1.0,0.55470,0.000000,0.002347
2,AwesomeScreenName,0.034873,2,0.000000,RabbaJabba,1.000000,1.0,0.27320,0.000000,0.002347
3,Awesomeuser90,0.015772,3,0.262568,KevinCarbonara,0.666667,1.0,0.91180,0.002978,0.021127
4,Awesomeuser90,0.015772,3,0.262568,RubiksSugarCube,0.666667,1.0,0.67050,0.002978,0.021127
...,...,...,...,...,...,...,...,...,...,...
92,Kuramhan,0.008204,45,0.000000,KevinCarbonara,-1.000000,0.0,-0.32390,0.000000,0.002347
93,NoExcuses1984,0.008135,46,0.000000,18650ll,-1.000000,0.0,-0.93740,0.000000,0.002347
94,RabbaJabba,0.010694,47,0.078558,SoulofZ,1.000000,0.0,-0.24840,0.001624,0.002347
95,guamisc,0.022063,48,0.000000,clemson07tigers,0.000000,0.0,-0.87115,0.004060,0.002347


In [68]:
GrafoWhithCentrality = GrafoWhithCentrality.drop(GrafoWhithCentrality.index[-1])
G = G.merge(GrafoWhithCentrality, left_on='Usuário', right_on='Usuário', how='left')

In [69]:
def create_subsetL(subset1):
  subsetL = subset1[subset1['Sc(i)'] <= 0]
  subsetL = subsetL[['Sc(i)']]
  return subsetL

def create_subsetR(subset2):
  subsetR = subset2[subset2['Sc(i)'] > 0]
  subsetR = subsetR[['Sc(i)']]
  return subsetR

In [70]:
def calc_SC(subsetL, subsetR):

  LC = abs(subsetL.sum() /  (len(subsetL) +  len(subsetR)))
  RC = abs(subsetR.sum() / (len(subsetR) +  len(subsetL)))
  SC = RC*LC
  return SC

In [71]:
def merge_subset(subset1, subset2, left_on, right_on):

  subset3 = subset1.merge(subset2, left_on=left_on, right_on=right_on, how='left')

  subset3 = subset3.drop(subset3.iloc[:, 11:],axis = 1)

  subset3 = subset3.drop_duplicates()

  return subset3

In [72]:
def BipPol_Calc(subset1, subset2, subset3, subset4):

  SW1 = -(2 * pow(subset3['p_x'] - 1/2, 2) + 1/2) * subset3['w_x']
  SW2 = -(2 * pow(subset4['p_x'] - 1/2, 2) + 1/2) * subset4['w_x']
  SW = (SW1 + SW2).sum()
  SWh = (SW/(len(SW1) + len(SW2))) + 2

  BipPol = calc_SC(create_subsetL(subset1), create_subsetR(subset2)) * SWh
  return BipPol

In [73]:
G['Fij*'] = (G['Betweenness Centrality'] + G['Engajamento'] + G['Influencia'])/3

In [74]:
from sklearn.model_selection import train_test_split

def BipPol(G):
    series_user = G['Usuário'].drop_duplicates()
    subset_user = pd.DataFrame(series_user)
    subset1, subset2 = train_test_split(subset_user, test_size=0.5, random_state=42)
    L = subset1.reset_index(drop=True)
    R = subset2.reset_index(drop=True)
    subset1 = subset1.merge(G, left_on='Usuário', right_on='Usuário', how='left')
    subset2 = subset2.merge(G, left_on='Usuário', right_on='Usuário', how='left')
    improving = True
    step = 0
    subset3 = merge_subset(subset1, subset2, 'Interação', 'Usuário')
    subset4 = merge_subset(subset2, subset1, 'Usuário', 'Interação')
    BipPol_old = BipPol_Calc(subset1, subset2, subset3, subset4)
    maxstep = pow((len(L.index) + len(R.index)), 2)
    while improving and step <= maxstep:
      improving = False
      for index, row in L.iterrows():
        aux = row['Usuário']
        user = subset1.query('@aux == Usuário')
        aux1 = subset1
        aux2 = subset2
        aux3 = merge_subset(aux1, aux2, 'Interação', 'Usuário')
        aux4 = merge_subset(aux2, aux1, 'Usuário', 'Interação')
        aux1 = aux1.drop(user.index)
        aux2 = pd.concat([aux2, user])
        BipPol_new = BipPol_Calc(aux1, aux2, aux3, aux4)
        if(BipPol_old['Sc(i)'] < BipPol_new['Sc(i)']):
          subset1 = aux1
          subset2 = aux2
          BipPol_old = BipPol_new
          R = pd.concat([R, pd.DataFrame([row], columns=R.columns)], ignore_index=True)
          L = L.drop(index).reset_index(drop=True)
          improving = True
          break
      for index, row in R.iterrows():
        aux = row['Usuário']
        user = subset2.query('@aux == Usuário')
        aux1 = subset2
        aux2 = subset1
        aux3 = merge_subset(aux1, aux2, 'Interação', 'Usuário')
        aux4 = merge_subset(aux2, aux1, 'Usuário', 'Interação')
        aux1 = aux1.drop(user.index)
        aux2 = pd.concat([aux2, user])
        BipPol_new = BipPol_Calc(aux1, aux2, aux3, aux4)
        if(BipPol_old['Sc(i)'] < BipPol_new['Sc(i)']):
          subset2 = aux1
          subset1 = aux2
          BipPol_old = BipPol_new
          L = pd.concat([L, pd.DataFrame([row], columns=L.columns)], ignore_index=True)
          R = R.drop(index).reset_index(drop=True)
          improving = True
          break

      step = step + 1
    L = L.merge(G, left_on='Usuário', right_on='Usuário', how='left')
    R = R.merge(G, left_on='Usuário', right_on='Usuário', how='left')
    return BipPol_old

In [75]:
import math

# G.sort_values(["Fij*", "Karma"], axis=0,ascending=False, inplace=True)
# BipPol_anterior = BipPol(G)
# min_value = 0
# max_value = 1
# meio = (min_value + max_value)/2

# for i in range(10):
#   series_user = G['Usuário'].drop_duplicates()
#   subset_user = pd.DataFrame(series_user)
#   G_linha = subset_user.head(math.floor(len(subset_user) * meio))
#   G_linha = G_linha.merge(G, left_on='Usuário', right_on='Usuário', how='left')
#   BipPol_atual = BipPol(G_linha)

#   if(BipPol_atual['Sc(i)'] > BipPol_anterior['Sc(i)']):
#     print(BipPol_atual)
#     print(BipPol_anterior)
#     print(meio)
#     print('-------------------')
#     meio = (meio + min_value)/2
#     BipPol_anterior = BipPol_atual
#   else:
#     print(BipPol_atual)
#     print(BipPol_anterior)
#     print(meio)
#     print('-------------------')
#     meio = (meio + max_value)/2





In [77]:
G.sort_values(["Fij*", "Karma"], axis=0,ascending=False, inplace=True)
BipPol_anterior = BipPol(G)
bipInit = BipPol_anterior
i = 5
while(i < len(G['Usuário'].drop_duplicates())):
  series_user = G['Usuário'].drop_duplicates()
  subset_user = pd.DataFrame(series_user)
  G_linha = subset_user.head(i)
  G_linha = G_linha.merge(G, left_on='Usuário', right_on='Usuário', how='left')
  BipPol_atual = BipPol(G_linha)

  if BipPol_atual['Sc(i)'] > BipPol_anterior['Sc(i)']:
    BipPol_anterior = BipPol_atual
    porcentagem = (len(G_linha) / len(G)) * 100
    print(bipInit)
    print(BipPol_atual)
    print(porcentagem)
    print('-------------------')

  i = i + 5


